In [1]:
import xmltodict
import pandas as pd

In [99]:
path = r"C:\Users\bekert\OneDrive - KPMG\Documents\python\wypakowywanie_sprawek_krs\e-sprawozdanie 2023 NS.xml"
with open(path,encoding="utf8") as f:
    data = xmltodict.parse(f.read())

In [101]:
def find_key(dict_:dict,search:str) ->str:
    for key in dict_.keys():
        if search in key:
            return key
        
    raise Exception("Brak szukanej frazy w slowniku")

dict_keys(['jin:RZiSKalk'])

In [106]:
jdn_inna = find_key(data,"JednostkaI")
bilans = find_key(data[jdn_inna],"Bilans")
aktywa = find_key(data[jdn_inna][bilans],"Aktywa")
pasywa = find_key(data[jdn_inna][bilans],"Pasywa")
pl_1=find_key(data[jdn_inna],"RZiS")
pl_2=find_key(data[jdn_inna][pl_1],"RZiS")

In [107]:

def clean_face(face):
    if ":" in face:
        return face.split(":")[-1]
    else:
        return face
        
def upacked_dict(nested_dict,face=""):
    for face0 in nested_dict.keys():
        if face:
            final_face = f"{face}|{clean_face(face0)}"
        else:
            final_face =  f"{clean_face(face0)}"
        if "Kwota" in face0:
            yield (f"{final_face}|{nested_dict[face0]}")
        else:
            yield from upacked_dict(nested_dict[face0],final_face)


In [108]:
def get_dataset(list,type):
    temp_dict = {"typKwota":[],"value":[],"face1":[],"face2":[],"face3":[],"face4":[],"face5":[],"face6":[]}
    for row in list:
        *faces, typKwota, value = row.split("|")
        faces = (faces+[""]*6)[:6]
        for i,face in enumerate(faces,1):
            temp_dict[f"face{i}"].append(face)
        temp_dict["typKwota"].append(typKwota)
        temp_dict["value"].append(value)
    df = pd.DataFrame(temp_dict)
    df['Type'] = type
    df['value'] = pd.to_numeric(df['value'])
    return df.pivot(index=['face1','face2','face3','face4','face5','face6','Type'],columns=['typKwota'],values='value').reset_index()

In [109]:
pl = list(upacked_dict(data[jdn_inna][pl_1][pl_2]))
bs_aktywa = list(upacked_dict(data[jdn_inna][bilans][aktywa]))
bs_pasywa = list(upacked_dict(data[jdn_inna][bilans][pasywa]))

In [110]:
df_bs_a = get_dataset(pl,"pl")

In [111]:
df_bs_a

typKwota,face1,face2,face3,face4,face5,face6,Type,KwotaA,KwotaB
0,A,,,,,,pl,8.704312e+08,1.007315e+09
1,A,A_I,,,,,pl,8.187429e+08,9.399368e+08
2,A,A_II,,,,,pl,5.168828e+07,6.737868e+07
3,A,A_J,,,,,pl,5.023366e+08,5.752679e+08
4,B,,,,,,pl,6.221346e+08,7.512264e+08
5,B,B_I,,,,,pl,5.812400e+08,6.984242e+08
6,B,B_II,,,,,pl,4.089468e+07,5.280212e+07
7,B,B_J,,,,,pl,3.882584e+08,4.629189e+08
8,C,,,,,,pl,2.482966e+08,2.560891e+08
9,D,,,,,,pl,1.214860e+08,1.335486e+08


In [112]:
data[jdn_inna]

{'@xmlns:etd': 'http://crd.gov.pl/xml/schematy/dziedzinowe/mf/2016/01/25/eD/DefinicjeTypy/',
 '@xmlns:dtsf': 'http://www.mf.gov.pl/schematy/SF/DefinicjeTypySprawozdaniaFinansowe/2018/07/09/DefinicjeTypySprawozdaniaFinansowe/',
 '@xmlns:jin': 'http://www.mf.gov.pl/schematy/SF/DefinicjeTypySprawozdaniaFinansowe/2018/07/09/JednostkaInnaStruktury',
 '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
 '@xsi:schemaLocation': 'http://www.mf.gov.pl/schematy/SF/DefinicjeTypySprawozdaniaFinansowe/2018/07/09/JednostkaInnaWZlotych https://www.gov.pl/documents/2034621/2182793/JednostkaInnaWZlotych(1)_v1-2.xsd',
 '@xmlns': 'http://www.mf.gov.pl/schematy/SF/DefinicjeTypySprawozdaniaFinansowe/2018/07/09/JednostkaInnaWZlotych',
 'Naglowek': {'dtsf:OkresOd': '2023-01-01',
  'dtsf:OkresDo': '2023-12-31',
  'dtsf:DataSporzadzenia': '2024-03-28',
  'jin:KodSprawozdania': {'@kodSystemowy': 'SFJINZ (1)',
   '@wersjaSchemy': '1-2',
   '#text': 'SprFinJednostkaInnaWZlotych'},
  'jin:WariantSprawozdania

In [ ]:
def find_values(values_dict):
    for face0 in values_dict.keys():
        for face1 in values_dict[face0].keys():
            if "Kwota" in face1:
                print(f"{face0}:{values_dict[face0][face1]}")
            else:
                for face2 in values_dict[face0][face1].keys():
                    if "Kwota" in face2:
                        print(f"{face0}|{face1}:{values_dict[face0][face1][face2]}")
                    else:
                        for face3 in values_dict[face0][face1][face2].keys():
                            if "Kwota" in face3:
                                print(f"{face0}|{face1}|{face2}:{values_dict[face0][face1][face2][face3]}")
                            else:
                                for face4 in values_dict[face0][face1][face2][face3].keys():
                                    print(f"{face0}|{face1}|{face2}|{face3}:{values_dict[face0][face1][face2][face3][face4]}")